# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy  as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('/datasets/insurance.csv')
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Приведем названия признаков к человеческому виду

In [3]:
df.columns = ['sex', 'age', 'income', 'famaly_members', 'claim']

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sex             5000 non-null   int64  
 1   age             5000 non-null   float64
 2   income          5000 non-null   float64
 3   famaly_members  5000 non-null   int64  
 4   claim           5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Проверим, за одно и избавимся от явных дублей

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sex             5000 non-null   int64  
 1   age             5000 non-null   float64
 2   income          5000 non-null   float64
 3   famaly_members  5000 non-null   int64  
 4   claim           5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [7]:
df.groupby(['claim']).agg({'claim':'count'})

,claim
claim,
0,4436
1,423
2,115
3,18
4,7
5,1


In [8]:
df[df["claim"]==4]

,sex,age,income,famaly_members,claim
1568,1,59.0,32100.0,0,4
2240,0,60.0,27900.0,0,4
3117,0,60.0,43400.0,2,4
3209,1,59.0,51700.0,2,4
3674,1,59.0,29600.0,0,4
3907,1,61.0,39600.0,1,4
4019,0,62.0,14100.0,0,4


В представленном ДФ было некоторое кол-во явных дубликатов, в остальном данные предподготовенные, полные. Страховые выплаты получают ~11% заявителей. Наиболее часто встречается 1 выплата, но дохоодит и до пяти.  

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$


Свойства матриц, необходимые для обоснования:

$(A+B)^T = A^T + B^T$

$(A^{-1})^T = (A^T)^{-1}$

$(AB)C = (AB)С$, важна последовательность умножения матриц

$AE = EA = A, где   Е - единичная  матрица$

$AA^{-1} = A^{-1}A = E, где   Е - единичная  матрица$

$(A^{-1})^{-1} = A$

$(A^T)^T = A$

$(AB)^T=B^T A^T$, Важно, что при раскрытии скобок матрицы меняются местами

$(AB)^{-1} = B^{-1} A^{-1}$, Важно, что при раскрытии скобок матрицы меняются местами

**Ответ:** качество линейной регрессии не изменится при умножении марицы признаков на произвольную обратимую марицу

**Обоснование:**

Формула предсказания:
    
$a=X(X^T X)^{-1} X^T y$

Умножим матрицу фич из этой формулы на случайную матрицу P. Получим:

$a' = XP((XP)^T XP)^{-1} (XP)^T y$

Вынесем P из скобок:

$a' = XP P^{-1}(X^T X)^{-1}(P^T)^{-1} P^T X^T y$

Из свойств матриц известно, что при перемоножени $P^{-1}$ и $P$,  $(P^T)^{-1}$ и $P^T$ получаются единичные матрицы $E$:

$a' = X E  (X^T X)^{-1} E X^T y$

При умнодении марицы на единичную, марица остается неизменной, таким образуо молучаем:

$a'=X(X^T X)^{-1} X^T y = a$ 

Ч.т.д

## Алгоритм преобразования

**Алгоритм**

1) обучаем линейную регресси на признаках без изменений

2) вычисляем метрику R2

3) создаем случайную матрицу, проверяем ее на обратимость

4) домножаем матрицу признаков на случайную матрицу P

5) обучаем линейную регресси на признаках, умноженных на случайную матрицу P

6) вычисляем метрику R2 для можифицированной регресси

7) сравниваем значения метрик, деаем выводы

### Проверка алгоритма

#### обучаем линейную регресси на признаках без изменений

In [9]:
features = df.drop(['claim'], axis=1)
target = df['claim'] 

features_train, features_valid, target_train, target_valid = train_test_split(
   features, target, test_size=0.25, random_state=12345) # отделите 25% данных для валидационной выборки
 
model = LinearRegression() # инициализируйте модель LinearRegression
model.fit(features_train, target_train) # обучите модель на тренировочной выборке
predictions_valid = model.predict(features_valid) # получите предсказания модели на валидационной выборке

#### вычислим r2 признаках без изменений

In [10]:
r2 =r2_score(target_valid, predictions_valid)
r2

0.43522757127026546

#### создаем случайную матрицу, проверяем ее на обратимость

In [11]:
np.random.seed(1) #зафиксируем гпсч
p = np.random.normal(size = (4,4)) # генерим матрицу, т.к. у фич 4 столбцов, то марица 4х4, для соблюдений праивла матричного умножения
display(p)
display(np.linalg.inv(p))

array([[ 1.62434536, -0.61175641, -0.52817175, -1.07296862],
       [ 0.86540763, -2.3015387 ,  1.74481176, -0.7612069 ],
       [ 0.3190391 , -0.24937038,  1.46210794, -2.06014071],
       [-0.3224172 , -0.38405435,  1.13376944, -1.09989127]])

array([[-0.35094377,  0.36251002,  1.39318587, -2.51802381],
       [-0.62956985, -0.08847157,  1.42691875, -1.9972879 ],
       [-0.93695498,  0.4429333 ,  1.44590393, -2.10075869],
       [-0.64311046,  0.38120375,  0.58380338, -1.63912083]])

#### домножаем матрицу признаков на случайную матрицу P

In [12]:
features_mod = df.drop(['claim'], axis=1)@p #непосредсвенно умнодаем фичи на матрицу P
target_mod = df['claim'] 

features_train_mod, features_valid_mod, target_train_mod, target_valid_mod = train_test_split(
   features_mod, target_mod, test_size=0.25, random_state=12345) # отделите 25% данных для валидационной выборки
 
model_mod = LinearRegression() # инициализируйте модель LinearRegression
model_mod.fit(features_train_mod, target_train_mod) # обучите модель на тренировочной выборке
predictions_valid_mod = model_mod.predict(features_valid_mod) # получите предсказания модели на валидационной выборке

#### вычисляем метрику R2 для можифицированной регресси

In [13]:
r2_mod =r2_score(target_valid_mod, predictions_valid_mod)
r2_mod

0.43522757127035283

***Вывод:***

В ходе данной работы обосновано и доказано на практике предположение о возможности шифрования исходных данных для обучения путем матричного умножения матрицы признаков на случайную матрицу. Обучены 2 можели линейной регресси: на открытых и на шифрованных данных. Метрики r2 обоих моделей совпадают из чиге можно сдлеать вывод, что подобная методика шифрования не влияет на качество моделей.